In [1]:
data = open('flare').read()
data = data.replace('\n', ' ').replace('\r', ' ')
print(data)

flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere. flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere. flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere. flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere. flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere. flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere. flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere. flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere. flare is a teacher in ai industry. He ob

In [3]:
letters = list(set(data))
num_letters = len(letters)
print(num_letters)
print(letters)

23
[' ', 'f', 'u', 't', 'b', 'a', 'y', 'c', 'o', 'A', 'r', 'd', 'H', 'n', 'p', 'S', 'i', 'm', 'e', 's', 'l', 'h', '.']


In [4]:
char_to_ix = { ch:i for i,ch in enumerate(letters) }
ix_to_char = { i:ch for i,ch in enumerate(letters) }
print(char_to_ix)

{' ': 0, 'f': 1, 'u': 2, 't': 3, 'b': 4, 'a': 5, 'y': 6, 'c': 7, 'o': 8, 'A': 9, 'r': 10, 'd': 11, 'H': 12, 'n': 13, 'p': 14, 'S': 15, 'i': 16, 'm': 17, 'e': 18, 's': 19, 'l': 20, 'h': 21, '.': 22}


In [5]:
time_step = 20

In [9]:
import numpy as np
from keras.utils import to_categorical


#滑动窗口提取数据
def extract_data(_data, slide):    
    x = []
    y = []    
    for i in range(len(_data) - slide):
        x.append([a for a in _data[i:i + slide]])
        y.append(_data[i + slide])        
    return x,y

#字符到数字的批量转化
def char_to_int_data(x, y, char_to_int):    
    x_to_int = []
    y_to_int = []
    for i in range(len(x)):
        x_to_int.append([char_to_int[char] for char in x[i]])
        y_to_int.append([char_to_int[char] for char in y[i]])    
    return x_to_int, y_to_int

#实现输入字符文章的批量处理，输入整个字符、滑动窗口大小、转化字典
def data_preprocessing(_data, slide, _num_letters, char_to_int):    
    char_data = extract_data(_data, slide)
    int_data = char_to_int_data(char_data[0], char_data[1], char_to_int)
    input_data = int_data[0]
    output = list(np.array(int_data[1]).flatten())
    input_reshaped = np.array(input_data).reshape(len(input_data), slide)
    new = np.random.randint(0, 10, size=[input_reshaped.shape[0], input_reshaped.shape[1], _num_letters])
    for i in range(input_reshaped.shape[0]):
        for j in range(input_reshaped.shape[1]):
            new[i,j,:] = to_categorical(input_reshaped[i,j], num_classes=_num_letters)
    return new, output


In [11]:
x, y = data_preprocessing(data, time_step, num_letters, char_to_ix)


In [13]:
print(x.shape)
print(len(y))

(56620, 20, 23)
56620


In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=10)

In [16]:
print(x_train.shape, len(y_train))

(50958, 20, 23) 50958


In [17]:
y_train_category = to_categorical(y_train, num_classes=num_letters)
print(y_train_category)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input

model = Sequential()
model.add(Input(shape=(time_step, num_letters)))
model.add(LSTM(20, activation='relu'))
model.add(Dense(num_letters, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 20)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 23)             │           483 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,003 (15.64 KB)

 Trainable params: 4,003 (15.64 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.fit(x_train, y_train_category, epochs=20, batch_size=1000)

Epoch 1/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8382 - loss: 0.6942
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.8670 - loss: 0.5216
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9501 - loss: 0.2890
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9226 - loss: 0.3028
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9710 - loss: 0.1274
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 1.0000 - loss: 0.0502
Epoch 7/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 1.0000 - loss: 0.0230
Epoch 8/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 1.0000 - loss: 0.0132
Epoch 9/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 1.0000 - loss: 0.0086
Epoch 10/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 1.0000 - loss: 0.0060
Epoch 11/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 1.0000 - loss: 0.0045
Epoch 12/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy:

In [29]:
y_train_predict = np.argmax(model.predict(x_train), axis=1)

1593/1593 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step


In [30]:
print(y_train_predict)

[ 2 21 18 ...  2 20 11]


In [34]:
y_train_predict_char = [ix_to_char[i] for i in y_train_predict]
print(y_train_predict_char)

['u', 'h', 'e', 'n', 'c', 'c', 'a', 'n', 'a', 'i', 'n', 'r', ' ', ' ', 'b', 'i', 'o', 'r', 't', ' ', 'A', ' ', 'o', 'h', 'd', 'y', 'p', ' ', 't', ' ', 'i', ' ', 'e', 'i', ' ', 't', ' ', 'o', 'r', '.', 'a', 'u', 'r', 'f', 'r', ' ', 'e', 'b', 'i', 't', 'm', 'A', 't', 'S', ' ', 'e', 'c', 't', ' ', 'e', 'l', 'i', 'n', 'd', ' ', ' ', 'n', 'r', 'n', 'i', ' ', 'r', ' ', 'i', ' ', 'y', 'h', 'o', ' ', 't', 's', ' ', 'e', 'p', 'r', 'r', 'i', 'r', 'l', 't', 'o', 'h', ' ', 't', 'h', 'r', ' ', 'h', 'i', 'n', ' ', 'a', 'a', ' ', ' ', ' ', 'c', ' ', 'A', '.', 'i', 'd', 'u', 'b', 'e', 'i', 'm', ' ', 'e', 'a', 'h', ' ', 'u', ' ', 'a', 'p', 'e', 'e', 'n', 'r', 'p', 'a', 'i', 'b', 'n', ' ', 'e', ' ', 'a', 'n', 'f', ' ', 'n', 'a', 'r', 'h', ' ', 'p', ' ', 'a', 'r', 'a', 's', 's', 'h', 'i', 'r', 'a', '.', 'e', ' ', 'e', 'e', 'e', 'y', '.', ' ', 't', 'e', ' ', 'd', ' ', 'u', 's', 't', 'n', ' ', ' ', ' ', ' ', 'u', 'e', 'e', 't', ' ', 'p', ' ', 'n', 'H', 'y', 'p', 'A', 'h', ' ', 'n', 'e', 'r', 't', 'h', ' ',

In [35]:
y_test_predict = np.argmax(model.predict(x_test), axis=1)

177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [36]:
from sklearn.metrics import accuracy_score

accuracy_train = accuracy_score(y_train, y_train_predict)
accuracy_test = accuracy_score(y_test, y_test_predict)
print(accuracy_train)
print(accuracy_test)

1.0
1.0


In [41]:
new_letters = 'flare is a teacher in ai industry. He obtained his phd in Australia.'
x_new, y_new = data_preprocessing(new_letters, time_step, num_letters, char_to_ix)
y_new_predict = np.argmax(model.predict(x_new), axis=1)
y_new_predict_char = [ix_to_char[i] for i in y_new_predict]
print(''.join(y_new_predict_char))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
n ai industry. He obtained his phd in Australia.
